In [1]:
#使用自己的IBMQ帳號
import qiskit as Q
#輸入API
Q.IBMQ.enable_account('a4e24fdeb13e506499504e55f916431b8941bea79adbc62684eed812500223e0c0e63d64452f78f981ba5b0902ac80d861200fccd3d04e1b4ffc4c7c4f930686')

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [2]:
from qiskit import *
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.providers.aer import QasmSimulator
from math import pi, sqrt, sin, cos
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os,sys
from pandas import *
import pandas as pd
import time as t

In [3]:
sample_shots = 1024
simulator = QasmSimulator()
x_list = []
M4_list = []
Mb4_list = []
time = []

def vqe_ansatz_1(params):
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    ansatz = QuantumCircuit(q,c)
    #GHZ-like state
    ansatz.h(0)
    ansatz.cx(0,1)
    ansatz.cx(1,2)
    ansatz.cx(2,3)
    ansatz.rz(params,q[0])
    return ansatz

def ex(vqe):
    sample_shots = 1024
    # Compile and execute
    # use real quantum computer
    provider = Q.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
    qcomp = provider.get_backend('ibm_perth') #called real quantum computer
    # calculation
    job_qasm = execute(vqe, backend = qcomp, initial_layout = [0,1,3,5], shots=sample_shots)
    result_sim = job_qasm.result().get_counts()
    # Calculate expectation
    get_0000=int(0 if result_sim.get('0000') is None else result_sim.get('0000'))
    get_0011=int(0 if result_sim.get('0011') is None else result_sim.get('0011'))
    get_0101=int(0 if result_sim.get('0101') is None else result_sim.get('0101'))
    get_0110=int(0 if result_sim.get('0110') is None else result_sim.get('0110'))
    get_1001=int(0 if result_sim.get('1001') is None else result_sim.get('1001'))
    get_1010=int(0 if result_sim.get('1010') is None else result_sim.get('1010'))
    get_1100=int(0 if result_sim.get('1100') is None else result_sim.get('1100'))
    get_1111=int(0 if result_sim.get('1111') is None else result_sim.get('1111'))
    get_0001=int(0 if result_sim.get('0001') is None else result_sim.get('0001'))
    get_0010=int(0 if result_sim.get('0010') is None else result_sim.get('0010'))
    get_0100=int(0 if result_sim.get('0100') is None else result_sim.get('0100'))
    get_0111=int(0 if result_sim.get('0111') is None else result_sim.get('0111'))
    get_1000=int(0 if result_sim.get('1000') is None else result_sim.get('1000'))
    get_1011=int(0 if result_sim.get('1011') is None else result_sim.get('1011'))
    get_1101=int(0 if result_sim.get('1101') is None else result_sim.get('1101'))
    get_1110=int(0 if result_sim.get('1110') is None else result_sim.get('1110'))
    
    P_0= get_0000 + get_0011 + get_0101 + get_0110 + get_1001 + get_1010 + get_1100 + get_1111
    P_1= get_0001 + get_0010 + get_0100 + get_0111 + get_1000 + get_1011 + get_1101 + get_1110
    return [P_0,P_1]

def ex1(vqe):
    sample_shots = 1024
    qobj = assemble(vqe,backend=QasmSimulator(),shots=sample_shots)
    es_job = QasmSimulator().run(qobj,backend_options={'method':'statevector'})
    result1 = es_job.result()
    counts = result1.get_counts(vqe)
    get_0000=int(0 if counts.get('0000') is None else counts.get('0000'))
    get_0011=int(0 if counts.get('0011') is None else counts.get('0011'))
    get_0101=int(0 if counts.get('0101') is None else counts.get('0101'))
    get_0110=int(0 if counts.get('0110') is None else counts.get('0110'))
    get_1001=int(0 if counts.get('1001') is None else counts.get('1001'))
    get_1010=int(0 if counts.get('1010') is None else counts.get('1010'))
    get_1100=int(0 if counts.get('1100') is None else counts.get('1100'))
    get_1111=int(0 if counts.get('1111') is None else counts.get('1111'))
    
    get_0001=int(0 if counts.get('0001') is None else counts.get('0001'))
    get_0010=int(0 if counts.get('0010') is None else counts.get('0010'))
    get_0100=int(0 if counts.get('0100') is None else counts.get('0100'))
    get_0111=int(0 if counts.get('0111') is None else counts.get('0111'))
    get_1000=int(0 if counts.get('1000') is None else counts.get('1000'))
    get_1011=int(0 if counts.get('1011') is None else counts.get('1011'))
    get_1101=int(0 if counts.get('1101') is None else counts.get('1101'))
    get_1110=int(0 if counts.get('1110') is None else counts.get('1110'))
    
    P_0= get_0000 + get_0011 + get_0101 + get_0110 + get_1001 + get_1010 + get_1100 + get_1111
    P_1= get_0001 + get_0010 + get_0100 + get_0111 + get_1000 + get_1011 + get_1101 + get_1110
    return [P_0,P_1]

In [4]:
def expval(params):
    def partial(params):
        #XYXX
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.h(0)
        vqe.h(1)
        vqe.sdg(2)
        vqe.h(2)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        XYXX = (ex_num[0]-ex_num[1])/sample_shots
        
        #YXXX
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.h(0)
        vqe.h(1)
        vqe.h(2)
        vqe.sdg(3)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        YXXX = (ex_num[0]-ex_num[1])/sample_shots
        
        #XXYX
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.h(0)
        vqe.sdg(1)
        vqe.h(1)
        vqe.h(2)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        XXYX = (ex_num[0]-ex_num[1])/sample_shots
        
        #YYYX
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.h(0)
        vqe.sdg(1)
        vqe.h(1)
        vqe.sdg(2)
        vqe.h(2)
        vqe.sdg(3)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        YYYX = (ex_num[0]-ex_num[1])/sample_shots
        
        #XYXY
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.sdg(0)
        vqe.h(0)
        vqe.h(1)
        vqe.sdg(2)
        vqe.h(2)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        XYXY = (ex_num[0]-ex_num[1])/sample_shots
        
        #YXXY
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.sdg(0)
        vqe.h(0)
        vqe.h(1)
        vqe.sdg(2)
        vqe.h(2)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        YXXY = (ex_num[0]-ex_num[1])/sample_shots
        
        #XXYY
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.sdg(0)
        vqe.h(0)
        vqe.sdg(1)
        vqe.h(1)
        vqe.h(2)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        XXYY = (ex_num[0]-ex_num[1])/sample_shots
        
        #YYYY
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.sdg(0)
        vqe.h(0)
        vqe.sdg(1)
        vqe.h(1)
        vqe.sdg(2)
        vqe.h(2)
        vqe.sdg(3)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        YYYY = (ex_num[0]-ex_num[1])/sample_shots
        
        #XYYX
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.h(0)
        vqe.sdg(1)
        vqe.h(1)
        vqe.sdg(2)
        vqe.h(2)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        XYYX = (ex_num[0]-ex_num[1])/sample_shots
        
        #YXYX
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.h(0)
        vqe.sdg(1)
        vqe.h(1)
        vqe.h(2)
        vqe.sdg(3)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        YXYX = (ex_num[0]-ex_num[1])/sample_shots
        
        #YYXX
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.h(0)
        vqe.h(1)
        vqe.sdg(2)
        vqe.h(2)
        vqe.sdg(3)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        YYXX = (ex_num[0]-ex_num[1])/sample_shots
        
        #XXXX
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.h(0)
        vqe.h(1)
        vqe.h(2)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        XXXX = (ex_num[0]-ex_num[1])/sample_shots
        
        #XYYY
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.sdg(0)
        vqe.h(0)
        vqe.sdg(1)
        vqe.h(1)
        vqe.sdg(2)
        vqe.h(2)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        XYYY = (ex_num[0]-ex_num[1])/sample_shots
        
        #YXYY
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.sdg(0)
        vqe.h(0)
        vqe.sdg(1)
        vqe.h(1)
        vqe.h(2)
        vqe.sdg(3)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        YXYY = (ex_num[0]-ex_num[1])/sample_shots
        
        #YYXY
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.sdg(0)
        vqe.h(0)
        vqe.h(1)
        vqe.sdg(2)
        vqe.h(2)
        vqe.sdg(3)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        YYXY = (ex_num[0]-ex_num[1])/sample_shots
        
        #XXXY
        vqe = vqe_ansatz_1(params)
        vqe.barrier()
        vqe.sdg(0)
        vqe.h(0)
        vqe.h(1)
        vqe.h(2)
        vqe.h(3)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
        vqe.measure([2],[2])
        vqe.measure([3],[3])
        ex_num = ex(vqe)
        XXXY = (ex_num[0]-ex_num[1])/sample_shots

        return [XYXX,YXXX,XXYX,YYYX,XYXY,YXXY,XXYY,YYYY,XYYX,YXYX,YYXX,XXXX,XYYY,YXYY,YYXY,XXXY]
    
    Y = partial(params) #叫出結果
    M4 = (Y[0]+Y[1]+Y[2]-Y[3])+(Y[4]+Y[5]+Y[6]-Y[7])+(Y[8]+Y[9]+Y[10]-Y[11])+(-Y[12]-Y[13]-Y[14]+Y[15])
    Mb4 = (Y[4]+Y[5]+Y[6]-Y[7])+(-Y[0]-Y[1]-Y[2]+Y[3])+(Y[8]+Y[9]+Y[10]-Y[11])+(Y[12]+Y[13]+Y[14]-Y[15])
    #normalized
    M4LR = 4
    M4ba = M4/M4LR
    Mb4ba = Mb4/M4LR
    t0 = t.time()
    time1 = (t.time() - t0)
    return [x, M4ba, Mb4ba, time1]

In [5]:
for x in np.arange (1.3, 2.05, 0.05):
    params= x*pi
    num = 3 #quantum circuit運轉5次 0135
    for i in range (num):
        X= expval(params)
        x_list.append(X[0])
        M4_list.append(X[1])
        Mb4_list.append(X[2])
        time.append(X[3])
        print(X)

dict = {'φ值': x_list, 'M4': M4_list, 'Mb4': Mb4_list, 'time cost': time}
df = pd.DataFrame(dict)
df.index = df.index+1
df.to_excel("4qmerminout.xlsx")

ibmqjob._wait_for_completion:ERROR:2023-05-10 19:01:51,095: Maximum retries exceeded: Error checking job status due to a network error.


IBMQJobApiError: 'Error checking job status due to a network error: "HTTPSConnectionPool(host=\'api.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/Network/ibm-q/Groups/open/Projects/main/Jobs/chdhcihcf94mmsdqb4o0/status/v/1 (Caused by NewConnectionError(\'<urllib3.connection.HTTPSConnection object at 0x000001F06FA67E50>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed\'))"'

In [ ]:
plt.title("t")
plt.xlabel("φ")
plt.ylabel("W")
plt.plot(x_list,M4_list,'.')